# A brief introduction to Python for Hydrological Analyses
This is a Jupyter Notebook, a web-based interactive development environment that allows to create and share python codes.
First things first, what is **Python**? Python is an high-level and general-purpose programming language. It can be used to write software in a wide variety of application domains, including hydrology. Python can be used to perform numerical calculations, statistical analyses or to access and plot data (even large datasets). <br>
In Jupyter Notebook the *Python shell* is embedded. The shell is where you can write and execute a line (or multiple lines) of code.
Python is open-source, and several packages are available covering many scientific and technological fields.

Let's start using Python as a **calculator**:

In [ ]:
190/3


If needed we can assign this result to a variable, and use the variable for further math or for other operations (such as converting to integer and priting it).

In [ ]:
result = 190/3
new_result = result - 14
int_result = int(new_result)
print(int(int_result))


What if we want to perform some more complex calculus? We can import the **math** package, loading several mathematical functions (such as the square root)

In [ ]:
import math
sqrt_result = math.sqrt(int_result)
print(sqrt_result)


What if we want to work not with a single value, but with a **vector** composed of multiple values?

In [ ]:
import numpy as np
array = [1,4,100,3,-2]
print(array)
print('------> complete array maximum: ' + str(np.max(array)))
array_nan = array
array_nan[2] = np.nan
print(array_nan)
print('------> incomplete array maximum: ' + str(np.max(array)))


Is the last result correct? Shouldn't be 4 the new maximum value? We can use a specific function for accounting for "Nan" or missing values: *np.nanmax* (part of numpy package)

In [ ]:
print('------> incomplete array maximum: ' + str(np.nanmax(array)))


### Dealing with Timeseries
Can we generate **timeseries** (multiple values with associated date) and plot it? Of course!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
ts = pd.Series(np.random.randn(365), index=pd.date_range('1/1/2010', periods=365))
plt.figure()
ts.plot(style='b-', label='Random timeseries')
plt.legend()


Finally, what if we want to read and plot an existing timeseries, available in **csv or txt format** for instance?

In [ ]:
temp_series = pd.read_csv('rainfall/Ogle_Aerdrome.txt', header=0, index_col=0, parse_dates=True, squeeze=True)
print(temp_series.head())
ax = temp_series.plot(style='b', title='rainfall at Ogle Airport')
ax.set_xlabel("")
ax.set_ylabel("Rainfall [mm]")
plt.show()


## Flood frequency analysis using Python
We can use python to compute flood statistics on a discharge timeseries. Reference to *hydro-informatics.github.io* <br>

Occurence of relevant (extreme) flood events can be expressed as **return period**, expressing the average recurrence interval of an event of a certain magnitude in units of time. It is the inverse of the **exceedance probability** (the likelihood of an event of a certain magnitude or higher).<br>
A significant assumption in calculating the return period is that individual events are assumed indipendent. This means that, for any given year, the probability of a 100-year flood occurring is 1/100.
Here below a table showing the recurrence intervals and related probabilities of occurrences.

| Return Period (years) | Annual exceeding probability (%) |
| --- | --- |
| 2 | 50 |
| 5 | 10 |
| 10 | 10 |
| 50 | 2 |
| 100 | 1 |
| 500 | 0.2 |

First things firt, we should import the **discharge data**. For instance the records on Potaro River at Kaieteur Falls. 

In [ ]:
df = pd.read_csv("discharge/Potaro_River_at_Kaieteur_Falls.txt",
                 header=0,
                 sep=",",
                 names=["Date", "Q"],
                 parse_dates=[0],
                 index_col=["Date"])
df['Q']=df['Q'].astype(float)
ax = df.plot(title='Potaro River at Kaieteur Falls')


This is the complete timeseries: we have to select only the **yearly maxima**. It is quite straightforward with *pandas dataframe*, we can resample our dataset (which has been indexed with dates).


In [ ]:
df_ymax = df.resample(rule="A", kind="period").max()
df_ymax["year"] = df_ymax.index.year
df_ymax.reset_index(inplace=True, drop=True)
#df_ymax = df_ymax.dropna()
print(df_ymax.head())
df_ymax.plot(kind='scatter',x='year',y='Q',color='red')
plt.show()


### Return period analysis
We should compute the exceedence probability *Pr*, and the resulting recurrence interval.
Pr is defined as: $Pr_{i} = \frac{(n-i+1)}{n+1}$\
Where *n* is the total number of observation years and *i* is the rank of the event.

In [ ]:
df_ymax_sorted = df_ymax.sort_values(by="Q")
n = df_ymax_sorted.shape[0]
df_ymax_sorted.insert(0, "rank", range(1, 1 + n))
print(df_ymax_sorted.head())


The **exceedence probability** ( *pr* ) can be calculated applying the formula:

In [ ]:
df_ymax_sorted["pr"] = (n - df_ymax_sorted["rank"] + 1) / (n + 1)
print(df_ymax_sorted.tail())


The **recurrence interval**( *return-period* ) is the inverse of the probability, thus:

In [ ]:
df_ymax_sorted["return-period"] = 1 / df_ymax_sorted["pr"]
print(df_ymax_sorted.tail())


Once create the table (*dataframe*) with all required information (**Probability** and **Return-Period**) we might plot it to visualise the recurrence interval of each observed discharge. It is worth mentioning that this analysis and the resulting plot refer only to observed values.<br>
To extrapolate recurrence interval beyond the observation period (the 1-in-100 years flood values, for instance) a prediction model is needed (Gumbel, GEV, etc..).

In [ ]:
df_ymax_sorted.plot.scatter(y="Q",
                         x="return-period",
                         title="Return period [years] ",
                         color='blue',
                         grid=True,
                         fontsize=14,
                         logy=False,
                         label="Sorted values")


## That is for this brief practical introduction to Python! 
### You might exercise a bit:

* in the virtual environment you should find other discharge .csv files. Ask python to read and analyse your selected input file!
* which is the discharge value for Return Period = 5?
* which is the discharge value with a Probability of exceedance = 0.5?
* can you plot the return-periods in a logaritmic scale (for y)?